# Módulo de Auditoría 16: Estrategia Editorial Basada en Datos (NLP)
## 🛡️ Contexto del Análisis
El mercado editorial es altamente competitivo. Se auditan miles de reseñas de lectores para decodificar los factores de éxito de los "Best Sellers" y proponer nuevas funcionalidades para la plataforma de venta.

## 🎯 Objetivos de la Auditoría
1.  **Minería de Opiniones:** Analizar qué valoran los lectores en sagas masivas.
2.  **Detección de Autoridad:** Identificar a los usuarios cuyas reseñas tienen mayor peso en la comunidad ("Superusuarios").
3.  **Propuesta de Valor:** Diseñar una estrategia de producto (Adquisición vs Retención) basada en los hallazgos textuales.


# **Proyecto Final de SQL: Análisis de la Base de Datos de Libros**

**Autor:** Octavio Landa Verde

**Fecha:** 23 de octubre del 2025

## **Parte I: Estructurando para el Éxito: Plan del Proyecto**

Este análisis adopta una **EDT (Estructura de Desglose del Trabajo) Basada en Fases** para asegurar un enfoque metodológico, robusto y profesional.

### **1.0 Comprensión del Negocio y Planificación**

**Objetivo del Negocio:** Una startup de aplicaciones de libros necesita analizar su base de datos para generar una propuesta de valor para un nuevo producto. Debemos identificar patrones de publicación, autores de alto rendimiento y comportamiento de los usuarios para informar la estrategia.

**Preguntas Clave (Tareas del Proyecto):**

1. ¿Cuántos libros de nuestro catálogo son "modernos" (publicados después del 1 de enero de 2000)?  
2. ¿Cuál es el engagement (reseñas y calificación) para cada libro?  
3. ¿Qué editoriales publican el contenido más "sustancial" (más de 50 páginas)?  
4. ¿Qué autores generan la mayor satisfacción en libros que ya tienen una tracción significativa (más de 50 calificaciones)?  
5. ¿Cuál es el comportamiento de nuestros "superusuarios" (aquellos con más de 50 calificaciones) en términos de reseñas de texto?

## **Parte II: 2.0 Adquisición y Preparación de Datos**

### **2.1 Conexión a la Base de Datos y Función Auxiliar**

Primero, importamos las bibliotecas necesarias (pandas y sqlalchemy) y establecemos la conexión con la base de datos PostgreSQL. Las credenciales se han extraído del PDF proporcionado.

In [ ]:
# 1.0 Importar y Conectar

# Importar bibliotecas
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la base de datos
db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

# Crear la cadena de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Crear el motor de conexión
# 'sslmode': 'require' es necesario para conectarse a este host en la nube
try:
    engine = create_engine(connection_string, connect_args={'sslmode': 'require'})
    print("¡Conexión a la base de datos exitosa!")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

# Definir una función auxiliar para ejecutar consultas
def run_query(query):
    """Ejecuta una consulta SQL y devuelve un DataFrame de pandas."""
    try:
        # pd.io.sql.read_sql está obsoleto, usamos pd.read_sql
        return pd.read_sql(query, con=engine)
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        return pd.DataFrame()


#### **Observación del Análisis:**
La conexión a la base de datos es el primer punto de fallo. Usamos **sqlalchemy** por ser el estándar de la industria en Python para interactuar con SQL, lo que nos permite ser agnósticos al motor de base de datos (PostgreSQL, MySQL, etc.).  
Pandas se usará como un "puente" para traer los resultados de las consultas SQL a un DataFrame y poder visualizarlos limpiamente en el notebook.  
Se incluye un bloque try-except para un manejo de errores profesional y una función auxiliar **run\_query** para mantener el código limpio (DRY \- Don't Repeat Yourself).

### **2.2 Inspección Inicial de Tablas (Análisis de Esquema)**

Antes de ejecutar las consultas del proyecto, realizamos una inspección LIMIT 5 de cada tabla. Esto es vital para confirmar que los nombres de las columnas y los tipos de datos coinciden con el esquema proporcionado.

**Esquema de la Base de Datos:**

* books: (book\_id, author\_id, title, num\_pages, publication\_date, publisher\_id)  
* authors: (author\_id, author)  
* publishers: (publisher\_id, publisher)  
* ratings: (rating\_id, book\_id, username, rating)  
* reviews: (review\_id, book\_id, username, text)

In [ ]:
# 2.2 Inspección de Tablas

print("--- Tabla: books (Primeras 5 filas) ---")
query_books = "SELECT * FROM books LIMIT 5;"
display(run_query(query_books))

print("\n--- Tabla: authors (Primeras 5 filas) ---")
query_authors = "SELECT * FROM authors LIMIT 5;"
display(run_query(query_authors))

print("\n--- Tabla: publishers (Primeras 5 filas) ---")
query_publishers = "SELECT * FROM publishers LIMIT 5;"
display(run_query(query_publishers))

print("\n--- Tabla: ratings (Primeras 5 filas) ---")
query_ratings = "SELECT * FROM ratings LIMIT 5;"
display(run_query(query_ratings))

print("\n--- Tabla: reviews (Primeras 5 filas) ---")
query_reviews = "SELECT * FROM reviews LIMIT 5;"
display(run_query(query_reviews))



--- Tabla: books (Primeras 5 filas) ---


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



--- Tabla: authors (Primeras 5 filas) ---


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



--- Tabla: publishers (Primeras 5 filas) ---


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



--- Tabla: ratings (Primeras 5 filas) ---


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



--- Tabla: reviews (Primeras 5 filas) ---


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## **Parte III: 3.0 Ejecución y Análisis (Respondiendo las Preguntas Clave)**

Aquí abordamos cada una de las 5 preguntas de negocio con una consulta SQL específica.

### **3.1 Tarea 1: Libros "Modernos" (Publicados post-2000)**

**Tarea:** Encuentra el número de libros publicados después del 1 de enero de 2000\.

In [ ]:
# 3.0 Tarea 1

query_1 = """
SELECT COUNT(book_id) AS total_books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""

print("--- Tarea 1: Libros publicados después del 1 de enero de 2000 ---")
df_1 = run_query(query_1)
display(df_1)

--- Tarea 1: Libros publicados después del 1 de enero de 2000 ---


,total_books_after_2000
0,819


#### **Observaciones y Conclusión (Tarea 1):**

* **Observaciones:** Se utiliza una consulta COUNT(\*) simple sobre la tabla books. La cláusula WHERE filtra publication\_date usando el formato de fecha estándar de SQL YYYY-MM-DD.  
* **Conclusión:** Este número define la "modernidad" del catálogo. Un número alto (cercano al total de libros) sugiere que la plataforma está enfocada en títulos contemporáneos, lo que atrae a un público que busca novedades. Un número bajo indicaría un enfoque en clásicos.

### **3.2 Tarea 2: Engagement por Libro (Reseñas y Calificación)**

**Tarea:** Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

*(Interpretación: "Reseñas de usuarios" se refiere al conteo de "calificaciones" (ratings), ya que esta es la métrica de engagement más directa y universal).*

In [ ]:
# 3.2 Tarea 2

query_2 = """
SELECT 
    b.title,
    COUNT(r.rating_id) AS num_ratings,
    AVG(r.rating) AS avg_rating
FROM 
    books AS b
LEFT JOIN 
    ratings AS r ON b.book_id = r.book_id
GROUP BY 
    b.book_id, b.title
ORDER BY 
    num_ratings DESC, avg_rating DESC
LIMIT 10;
"""

print("--- Tarea 2: Top 10 libros por número de reseñas y calificación promedio ---")
df_2 = run_query(query_2)
display(df_2)


--- Tarea 2: Top 10 libros por número de reseñas y calificación promedio ---


,title,num_ratings,avg_rating
0,Twilight (Twilight #1),160,3.662500
1,The Hobbit or There and Back Again,88,4.125000
2,The Catcher in the Rye,86,3.825581
3,Angels & Demons (Robert Langdon #1),84,3.678571
4,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.414634
5,Harry Potter and the Chamber of Secrets (Harry...,80,4.287500
6,Harry Potter and the Order of the Phoenix (Har...,75,4.186667
7,The Fellowship of the Ring (The Lord of the Ri...,74,4.391892
8,Animal Farm,74,3.729730
9,Harry Potter and the Half-Blood Prince (Harry ...,73,4.246575


#### **Observaciones y Conclusión (Tarea 2):**

* **Observaciones:** Se utiliza un LEFT JOIN desde books hacia ratings. Esto es metodológicamente crucial para asegurar que los libros *sin calificaciones* no sean excluidos (aparecerían con 0 num\_ratings). Agrupamos por book\_id (la clave primaria) y title (para visualización). Ordenamos por popularidad (num\_ratings) para ver los "best-sellers" primero.  
* **Conclusión:** Esta consulta identifica los activos más valiosos de la plataforma. Los libros con alto num\_ratings son los motores de tráfico. Los libros con alto avg\_rating (especialmente si tienen un num\_ratings moderado) son los productos de mayor calidad y deben ser recomendados activamente para construir la reputación de la marca.

### **3.3 Tarea 3: Editoriales con Contenido "Sustancial"**

**Tarea:** Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (para excluir folletos y publicaciones similares).

In [ ]:
# 3.3 Tarea 3

query_3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS num_books
FROM 
    publishers AS p
JOIN 
    books AS b ON p.publisher_id = b.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher
ORDER BY 
    num_books DESC
LIMIT 5;
"""

print("--- Tarea 3: Top 5 Editoriales con más libros (> 50 páginas) ---")
df_3 = run_query(query_3)
display(df_3)


--- Tarea 3: Top 5 Editoriales con más libros (> 50 páginas) ---


,publisher,num_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


#### **Observaciones y Conclusión (Tarea 3):**

* **Observaciones:** Se unen publishers y books. El filtro WHERE b.num\_pages \> 50 se aplica *antes* de la agrupación. Esta es una práctica de optimización de SQL: filtrar primero reduce el conjunto de datos sobre el cual se deben realizar los cálculos de agregación (COUNT).  
* **Conclusión:** Esta consulta identifica a los socios de negocio estratégicos. La editorial en la cima de esta lista es la principal proveedora de "libros reales" (no folletos). Para la startup, esta es la editorial con la que se deben fortalecer lazos, negociar promociones y asegurar que su catálogo esté completo en la plataforma.

### **3.4 Tarea 4: Autores de Mayor Calificación (Tracción Significativa)**

**Tarea:** Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [ ]:
# 3.4 Tarea 4

query_4 = """
WITH PopularBooks AS (
    -- Paso 1: Encontrar libros con 50+ calificaciones y su calificación promedio
    -- (Estos son los libros con tracción estadística)
    SELECT 
        book_id, 
        AVG(rating) AS avg_book_rating
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50
),
AuthorBookRatings AS (
    -- Paso 2: Unir esos libros populares con sus autores
    SELECT
        b.author_id,
        pb.avg_book_rating
    FROM PopularBooks pb
    JOIN books b ON pb.book_id = b.book_id
)
-- Paso 3: Calcular el promedio de las calificaciones de los libros por autor
-- (Buscamos autores consistentemente buenos)
SELECT
    a.author,
    AVG(abr.avg_book_rating) AS final_avg_author_rating,
    COUNT(abr.avg_book_rating) AS num_popular_books
FROM AuthorBookRatings abr
JOIN authors a ON abr.author_id = a.author_id
GROUP BY a.author
ORDER BY final_avg_author_rating DESC
LIMIT 5;
"""

print("--- Tarea 4: Top 5 Autores por calificación promedio (en libros con +50 reseñas) ---")
df_4 = run_query(query_4)
display(df_4)


--- Tarea 4: Top 5 Autores por calificación promedio (en libros con +50 reseñas) ---


,author,final_avg_author_rating,num_popular_books
0,J.K. Rowling/Mary GrandPré,4.283844,4
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,1
2,J.R.R. Tolkien,4.258446,2
3,Louisa May Alcott,4.192308,1
4,Rick Riordan,4.080645,1


#### **Observaciones y Conclusión (Tarea 4):**

* **Observaciones:** Esta consulta es la más compleja y utiliza WITH (Common Table Expressions o CTEs) para dividir el problema en pasos lógicos:  
  1. **PopularBooks:** Primero, aislamos solo los libros que son estadísticamente relevantes. Un libro con 1 calificación de 5.0 no es fiable. Un libro con 50+ calificaciones sí lo es. Usamos HAVING para filtrar *después* de la agregación.  
  2. **AuthorBookRatings:** Vinculamos estos libros probados con sus autores.  
  3. **Consulta Final:** Agrupamos por autor y calculamos el AVG() del avg\_book\_rating de sus libros populares.  
* **Conclusión:** Esta lista es oro. No muestra al autor con *un* solo éxito, sino al autor que **consistentemente produce libros populares y aclamados por la crítica**. Estos son los autores "más seguros" para promocionar. La propuesta de valor del nuevo producto podría ser una "Colección del Autor Destacado" basada en el ganador de esta consulta.

### **3.5 Tarea 5: Comportamiento de "Superusuarios"**

**Tarea:** Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [ ]:
# 3.5 Tarea 5

query_5 = """
WITH ActiveRaters AS (
    -- Paso 1: Encontrar usuarios que calificaron > 50 libros
    -- (Estos son los usuarios 'power users' o 'activos')
    SELECT 
        username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
ReviewCountsPerRater AS (
    -- Paso 2: Contar cuántas reseñas de texto tiene CADA uno de esos usuarios
    SELECT
        ar.username,
        COUNT(r.review_id) AS num_text_reviews
    FROM ActiveRaters ar
    LEFT JOIN reviews r ON ar.username = r.username -- LEFT JOIN por si un súper-calificador no dejó reseñas de texto
    GROUP BY ar.username
)
    -- Paso 3: Calcular el promedio de esos conteos
    SELECT
    AVG(rc.num_text_reviews) AS avg_text_reviews_from_active_raters
    FROM ReviewCountsPerRater rc;
    """

print("--- Tarea 5: Promedio de reseñas de texto de usuarios 'activos' (+50 calificaciones) ---")
df_5 = run_query(query_5)
display(df_5)


--- Tarea 5: Promedio de reseñas de texto de usuarios 'activos' (+50 calificaciones) ---


,avg_text_reviews_from_active_raters
0,24.333333


<div class="alert alert-block alert-success">
<b>Paso 2</b> <a class="tocSkip"></a><br>
Has realizado un buen trabajo al formular una consulta SQL para contar con:
    
    - el número de libros publicados después de una fecha específica.
    - Calificación promedio
    - Editorial que ha publicado la mayor cantidad de libros
    - Autor con la calificación promedio más alta
    - Promedio de reseñas
    
Esto demuestra tu habilidad para manipular y consultar bases de datos de manera efectiva. Solamente para terminar de complementar te recomiendo incluir el número de libros evaluados por autor para proporcionar un contexto adicional sobre la consistencia de sus calificaciones.
</div>

#### **Observaciones y Conclusión (Tarea 5):**

* **Observaciones:** De nuevo, los CTEs simplifican la lógica:  
  1. **ActiveRaters:** Identificamos el cohorte de "superusuarios" basándonos en su actividad de *calificación* (bajo esfuerzo).  
  2. **ReviewCountsPerRater:** Medimos su actividad de *reseña de texto* (alto esfuerzo). El LEFT JOIN es clave para incluir a los superusuarios que califican mucho pero escriben poco (contarían como 0 num\_text\_reviews).  
  3. **Consulta Final:** Calculamos el AVG() de estos conteos.  
* **Conclusión:** Esta métrica es un **Índice de Engagement de la Comunidad (CEI)**. Nos dice: "De nuestros usuarios más leales, ¿cuántos están dispuestos a generar contenido valioso (reseñas de texto)?"  
  * **Si el promedio es alto (ej. \> 10):** La comunidad está saludable y genera contenido orgánico que ayuda a vender.  
  * **Si el promedio es bajo (ej. \< 2):** Hay una desconexión. Los usuarios usan la app, pero no se sienten motivados a contribuir. La "propuesta de valor" del nuevo producto debería incluir funciones sociales o de gamificación para incentivar la escritura de reseñas.

## **Parte IV: 4.0 Conclusión General y Propuesta de Valor**

Este análisis SQL ha sentado las bases para una estrategia de producto basada en datos.

# **Propuesta Final de Producto: "Curación de Calidad y Comunidad Activa"**

**A:** Equipo de Producto y Marketing

**De:** Octavio Landa Verde (Analista de Datos)

**Fecha:** 23 de octubre del 2025

**Asunto:** Propuesta de Valor para el Nuevo Producto basada en el análisis de la Base de Datos

## **1\. Resumen Ejecutivo**

El análisis de nuestra base de datos revela una oportunidad clara. Nuestro catálogo es predominantemente moderno (**819 libros** post-2000) y, lo más importante, tenemos una base de "superusuarios" increíblemente activa y comprometida (promedio de **24.33 reseñas de texto** por usuario activo).

La estrategia para el nuevo producto no debe ser tratar de *crear* engagement, sino **amplificar el engagement que ya existe**. Propongo un enfoque de producto centrado en dos pilares:

1. **Curación de Alta Calidad:** Enfocarse en nuestros autores y socios probados.  
2. **Capital Social:** Aprovechar el contenido generado por nuestros superusuarios.

## **2\. Propuestas de Implementación (Basadas en Hallazgos)**

A continuación se detallan las acciones recomendadas basadas en los resultados de cada consulta.

### **Propuesta A: Implementar el "Doble Embudo" de Marketing (Basado en Tarea 2\)**

* **Hallazgo:** Los libros más populares no son necesariamente los de mayor calidad.  
  * **Motor de Tráfico:** `Twilight` (160 reseñas, 3.66 avg) genera la mayor cantidad de interacciones, pero tiene una calificación mediocre.  
  * **Joyas de Calidad:** La saga `Harry Potter` y `The Hobbit` tienen un volumen de reseñas muy alto (73-88) *y* calificaciones estelares (4.12 \- 4.41).  
* **Acción de Marketing (Adquisición):**  
  * Utilizar `Twilight` y `Angels & Demons` (3.67 avg) en campañas de adquisición de usuarios (ej. anuncios en redes sociales). Su alta popularidad es excelente para atraer clics a bajo costo.  
* **Acción de Producto (Retención):**  
  * Una vez que un usuario se registra atraído por `Twilight`, la *primera* pantalla de la app debe presentarle un módulo de "Los Favoritos de la Crítica", destacando a `Harry Potter` y `The Hobbit`. Esto transfiere inmediatamente al usuario de un libro "divisivo" a uno de "calidad probada", aumentando la retención y la confianza en la marca.

### **Propuesta B: Foco en "Autores-Franquicia" y Socios Clave (Basado en Tareas 3 y 4\)**

* **Hallazgo:** El éxito en nuestra plataforma está concentrado.  
  * **Autores:** `J.K. Rowling` (4.28 avg en 4 libros populares) y `J.R.R. Tolkien` (4.25 avg en 2 libros populares) son nuestros autores más consistentes y aclamados.  
  * **Socios:** `Penguin Books` (42 libros sustanciales) y `Vintage` (31) son, por mucho, nuestros socios de contenido más importantes.  
* **Acción de Producto:**  
  * Desarrollar una *feature* destacada en la página de inicio: **"Colecciones de Autor"**. Empezar con "El Mundo Mágico de J.K. Rowling" y "La Tierra Media de Tolkien". Estos autores son apuestas seguras.  
* **Acción de Negocios (BD):**  
  * Iniciar conversaciones estratégicas con `Penguin Books` y `Vintage`. Proponerles ser "Socio Editorial del Mes" en la app, dándoles visibilidad a cambio de acceso anticipado a nuevos títulos o promociones exclusivas.

### **Propuesta C: De la Gamificación al "Capital Social" (Basado en Tarea 5\)**

* **Hallazgo:** Nuestros superusuarios están **extremadamente comprometidos**. Un promedio de **24.33 reseñas de texto** por cada usuario que ha calificado más de 50 libros es un número extraordinariamente alto.  
* **Acción de Producto (Crítica):**  
  * **NO INVERTIR** recursos en funciones de *gamificación* (insignias, puntos) para *incentivar* la escritura de reseñas. Nuestros usuarios más valiosos **ya lo están haciendo gratis**.  
  * **SÍ INVERTIR** recursos en funciones de **Capital Social** para *amplificar* el contenido que ya generan.  
    1. **Prioridad de Reseñas:** Modificar el algoritmo de reseñas para que las reseñas escritas por "Superusuarios" (aquellos con \>50 calificaciones) aparezcan siempre primero.  
    2. **Etiqueta "Superusuario":** Añadir una etiqueta visual "Superusuario" o "Crítico Verificado" junto a sus nombres para darles estatus y aumentar la confianza en sus reseñas.  
    3. **Perfiles de Usuario Avanzados:** Permitir que los usuarios sigan a estos "Superusuarios", convirtiéndolos en micro-influencers dentro de la propia plataforma.

## **3\. Próximos Pasos (Resumen)**

1. **Equipo de Marketing:** Implementar la estrategia del "Doble Embudo" (Adquisición con `Twilight`, Retención con `Harry Potter`).  
2. **Equipo de Producto:** Priorizar el desarrollo de las *features* "Colecciones de Autor" (Rowling, Tolkien) y la "Prioridad de Reseñas de Superusuarios".  
3. **Equipo de Negocios:** Contactar a `Penguin Books` para una alianza estratégica.